In [1]:
import ibis
from pyspark.sql import SparkSession

In [2]:

session = SparkSession.builder.getOrCreate()
ipsc = ibis.pyspark.connect(session)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/24 20:39:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/24 20:39:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
ipsc

In [4]:
ipsc.list_databases()

['default']

In [21]:
ipsc.list_tables()


['tmp_df_table']

In [23]:
my_table = ipsc.table("tmp_df_table")

In [26]:
for col in my_table.columns:
    if col[0] == "d":
        print(col)

derived_msa_md
derived_loan_product_type
derived_dwelling_category
derived_ethnicity
derived_race
derived_sex
discount_points
debt_to_income_ratio
denial_reason_1
denial_reason_2
denial_reason_3
denial_reason_4


In [6]:
df_rdd = session.read.csv("../data/sample_big_mortgage_file.csv.gz", header=True, sep="|")

In [7]:
df_rdd.count()

5

In [14]:
type(df_rdd)

pyspark.sql.dataframe.DataFrame

In [17]:
#df_rdd.write.mode("overwrite").saveAsTable("testdb.test")

df_rdd.createTempView("tmp_df_table")

In [18]:
query = "SELECT * FROM tmp_df_table"

In [19]:
df1 = session.sql(query)

In [20]:
df1.head()

Row(activity_year='2020', lei='KB1H1DSPRFMYMCUFXT09', derived_msa_md='47260', state_code='VA', county_code='51810', census_tract='51810046015', conforming_loan_limit='C', derived_loan_product_type='Conventional:First Lien', derived_dwelling_category='Single Family (1-4 Units):Site-Built', derived_ethnicity='Joint', derived_race='White', derived_sex='Joint', action_taken='3', purchaser_type='0', preapproval='2', loan_type='1', loan_purpose='2', lien_status='1', reverse_mortgage='2', open_end_line_of_credit='1', business_or_commercial_purpose='2', loan_amount='35000', combined_loan_to_value_ratio='NA', interest_rate='NA', rate_spread='NA', hoepa_status='3', total_loan_costs='NA', total_points_and_fees='NA', origination_charges='NA', discount_points='NA', lender_credits='NA', loan_term='361', prepayment_penalty_term='NA', intro_rate_period='NA', negative_amortization='2', interest_only_payment='2', balloon_payment='2', other_nonamortizing_features='2', property_value='175000', constructio